In [1]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
from sub import subMNIST       # testing the subclass of MNIST dataset

Files already downloaded
3000
750


# Pre Training Unsupervised

In [3]:
trainset_import = pickle.load(open("train_labeled.p", "rb"))
validset_import = pickle.load(open("validation.p", "rb"))
trainset_unl = pickle.load(open("train_unlabeled.p", "rb"))

In [48]:
train_loader = torch.utils.data.DataLoader(trainset_import, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)
train_unl_loader = torch.utils.data.DataLoader(trainset_unl, batch_size=64, shuffle=True)
#train_loader.dataset.train_data[0]
#train_loader.dataset.train_labels[0]
#train_loader.dataset.train_data[0]

In [5]:
import numpy

class Noise():
    def SaltAndPepper(self, X, rate=0.3):
        # Salt and pepper noise
        X = X.numpy()
        drop = X.shape[1]
        numpy.random.shuffle(drop)
        #drop = X.size()[1]
        #input = Variable(torch.randn(1, 1, 28, 28))
        sep = int(len(drop)*rate)
        drop = drop[:sep]
        X[:, drop[:sep/2]]=0
        X[:, drop[sep/2:]]=1
        X = torch.from_numpy(X)
        return X
        
    def GaussianNoise(self, X, sd=0.5):
        # Injecting small gaussian noise
        X = X.numpy()
        X += numpy.random.normal(0, sd, X.shape)
        X = torch.from_numpy(X)
        return X
        
    def MaskingNoise(self, X, rate=0.5):
        mask = (numpy.random.uniform(0,1, X.shape)<rate).astype("i4")
        X = mask*X
        return X
        
def SaltAndPepper(rate=0.3):
    # Salt and pepper noise
    def func(X):
        drop = numpy.random.uniform(0,1, X.shape)
        z = numpy.where(drop < 0.5*rate)
        o = numpy.where(numpy.abs(drop - 0.75*rate) < 0.25*rate)
        X[z]=0
        X[o]=1   
        return X
    return func
    
def GaussianNoise(self, sd=0.5):
    # Injecting small gaussian noise
    def func(X):
        X += numpy.random.normal(0, sd, X.shape)
        return X
    return func

Noise = Noise()

In [6]:
class DenoisingAutoencoder(nn.Module):
    """
    A denoising autoencoder will corrupt an input (add noise) and try to reconstruct it.
    """
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        # Define some model hyperparameters to work with MNIST images!
        input_size  = [784, 20] # dimensions of image
        out_size = [20,784]
        hidden_size = 1000  # number of hidden units - generally bigger than input size for DAE

        # Now, define the symbolic input to the model (Theano)
        # We use a matrix rather than a vector so that minibatch processing can be done in parallel.
        #x = T.fmatrix('x')
        #self.inputs = [x]
        self.fc1 = nn.Linear(input_size[0], input_size[1])
        self.fc2 = nn.Linear(out_size[0],out_size[1])

        # Build the model's parameters - a weight matrix and two bias vectors
        #self.W  = nn.(input_size[0],input_size[1])

#         # Perform the computation for a denoising autoencoder!
#         # first, add noise (corrupt) the input
#         corrupted_input = Noise.GaussianNoise(x, 0.5)
#         # next, compute the hidden layer given the inputs (the encoding function)
#         h1 = torch.tanh(torch.dot(corrupted_input, W))
#         # finally, create the reconstruction from the hidden layer (we tie the weights with W.T)
#         self.rec = torch.sigmoid(torch.dot(h1, W.T))
#         # the training cost is reconstruction error - with MNIST this is binary cross-entropy
#         self.train_cost = torch.binary_crossentropy(self.rec, x)

        # Compile everything into a Theano function for prediction!
        # When using real-world data in predictions, we wouldn't corrupt the input first.
        # Therefore, create another version of the hiddens and reconstruction without adding the noise
#         hiddens_predict = torch.tanh(torch.add(torch.mul(x, W),b1))
#         recon_predict   = torch.sigmoid(torch.add(torch.mul(h1, y.T),b2))
#         self.output     = recon_predict
        
    def decode(self, z, y):
        #rec = torch.sigmoid(torch.cat((z, y)+b2))  # concatenate z and y
        return self.rec
    
    def get_train_cost(self):
        #torch.binary_crossentropy(self.rec)
        return self.train_cost
    
    def get_inputs(self):
        return self.inputs

    # def get_params(self):
    #    return self.params

#     def get_outputs(self):
#         return self.output
    
    def forward(self, x):
        # Perform the computation for a denoising autoencoder!
        # first, add noise (corrupt) the input
        #corrupted_input = self.Noise.GaussianNoise(x, 0.5)
        # next, compute the hidden layer given the inputs (the encoding function)
        x = x.data
        x = x.view(-1,784)
        #print(x)
        corrupted_x = Noise.GaussianNoise(x, 0.5)
        corrupted_x = Variable(corrupted_x )
        x = Variable(x)
        h1 = F.sigmoid(self.fc1(corrupted_x))
        #print(h1)
        # finally, create the reconstruction from the hidden layer (we tie the weights with W.T)
        self.rec = F.sigmoid(self.fc2(h1))
        
        #print(self.rec)
        self.train_cost = F.binary_cross_entropy(self.rec, x)
        #print(self.train_cost)
        return self.rec
        # the training cost is reconstruction error - with MNIST this is binary cross-entropy
#         self.train_cost = torch.binary_crossentropy(self.rec, x)
#         hiddens_predict = torch.tanh(self.fc1(x))
#         recon_predict   = torch.sigmoid(self.fc1(hiddens_predict))
#         self.output     = recon_predict
         


In [7]:
dae = DenoisingAutoencoder()

In [8]:

#w = torch.zeros(278,20)
#w = Variable(w)
def train(epoch):
    dae.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #print(data)
        #data = Noise.GaussianNoise(data, 0.5)
        data, target = Variable(data), Variable(target)
        output = dae(data)
        loss = dae.get_train_cost()
        #print(loss)
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    dae.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = dae(data)
        test_loss += dae.get_train_cost().data[0]
        pred = output.data # get the index of the max log-probability
        #print(pred)
        data = data.view(-1,784)
        #print(data.data)
        correct += pred.eq(data.data).cpu().sum()
#         ind = output.data.max(1)[1]
#         actual = data.data.numpy()
#         if ind == actual:
#             correct += 1

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [9]:
# for batch_idx, (data, target) in enumerate(train_loader):
#     print(data)
#print(train_loader.dataset.train_data[0])

In [10]:
tt = train_unl_loader.dataset.train_data[0]
Variable(tt)

Variable containing:

Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     2
    0     0     0     0     0     0     0     0     0     0     0    14   194
    0     0     0     0     0     0     0     0     0     0     0    43   196
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0  

In [78]:
def train(epoch):
    dae.train()
    #output = np.array([None]*47000)
    output = []
    for batch_idx, _ in enumerate(train_unl_loader.dataset.train_data):
        #print(data)
        #data = Noise.GaussianNoise(data, 0.5)
        data = Variable(train_unl_loader.dataset.train_data[batch_idx].float())
        recx = dae(data).data.view(-1,1,28,28)
        output.append(recx.numpy())
        loss = dae.get_train_cost()
        #print(loss)
        optimizer.step()
        #if batch_idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx, len(train_unl_loader.dataset),
#                 100. * batch_idx/len(train_unl_loader.dataset), loss.data[0]))
    return output



In [79]:
optimizer = optim.SGD(dae.parameters(), lr=0.01, momentum=0.5)####tune to improve
criterion = nn.CrossEntropyLoss()

In [117]:
for epoch in range(1, 2):
    output = train(epoch)
    #test(epoch, valid_loader)

In [81]:
len(output)

47000

In [119]:
output = np.array(output)


In [121]:
output = torch.from_numpy(output)

In [122]:
import matplotlib.pyplot as plt
import torchvision
%matplotlib inline
def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
# plt.imshow(output[0])
dataiter = iter(output)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))

AttributeError: 'map' object has no attribute 'next'

In [86]:
rec_unl_loader = torch.utils.data.DataLoader(output, batch_size=64, shuffle=True)


In [87]:
type(train_unl_loader.dataset),type(rec_unl_loader.dataset)

(sub.subMNIST, torch.FloatTensor)

In [88]:
rec_unl_loader.dataset[1]


(0 ,0 ,.,.) = 

Columns 0 to 8 
   0.4982  0.4514  0.3015  0.4083  0.3829  0.5500  0.4334  0.5086  0.4684
  0.2287  0.5547  0.5381  0.4233  0.6115  0.6136  0.2904  0.5718  0.4826
  0.5525  0.6700  0.6324  0.5423  0.4645  0.6429  0.5358  0.6280  0.4171
  0.3916  0.6583  0.6712  0.6460  0.4110  0.6812  0.3656  0.5168  0.4676
  0.3646  0.4956  0.4400  0.6242  0.7213  0.3757  0.7069  0.5899  0.6013
  0.4192  0.5719  0.3901  0.5345  0.5284  0.6441  0.5046  0.5397  0.4160
  0.5534  0.6092  0.5978  0.6490  0.3634  0.4754  0.5372  0.3930  0.5708
  0.4739  0.4341  0.2849  0.5558  0.2873  0.4786  0.4012  0.6506  0.4884
  0.4363  0.3289  0.5188  0.4698  0.3814  0.6083  0.5868  0.3746  0.5309
  0.3861  0.5343  0.3776  0.3565  0.5170  0.5617  0.4711  0.4798  0.4316
  0.4309  0.3999  0.6717  0.3832  0.5260  0.3638  0.6046  0.4092  0.5032
  0.5163  0.4687  0.3069  0.4846  0.3419  0.4203  0.5063  0.3722  0.2879
  0.5059  0.3571  0.6907  0.5270  0.5415  0.6046  0.3679  0.4495  0.3620
  0.5771  0.6505 

In [35]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):####can improve
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)

model = Net()

In [36]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)####tune to improve
criterion = nn.CrossEntropyLoss()

In [37]:
# CPU only training

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #data = Noise.GaussianNoise(data)#add adaptive Gaussian Noises
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [38]:
for epoch in range(1, 11):
    train(epoch)
    test(epoch, valid_loader)

Train Epoch: 1 [0/3000 (0%)]	Loss: 2.286879
Train Epoch: 1 [640/3000 (21%)]	Loss: 2.334832
Train Epoch: 1 [1280/3000 (43%)]	Loss: 2.309296
Train Epoch: 1 [1920/3000 (64%)]	Loss: 2.283294
Train Epoch: 1 [2560/3000 (85%)]	Loss: 2.300826

Test set: Average loss: 2.2715, Accuracy: 2198/10000 (22%)

Train Epoch: 2 [0/3000 (0%)]	Loss: 2.302184
Train Epoch: 2 [640/3000 (21%)]	Loss: 2.272326
Train Epoch: 2 [1280/3000 (43%)]	Loss: 2.263016
Train Epoch: 2 [1920/3000 (64%)]	Loss: 2.286039
Train Epoch: 2 [2560/3000 (85%)]	Loss: 2.266297

Test set: Average loss: 2.1984, Accuracy: 3739/10000 (37%)

Train Epoch: 3 [0/3000 (0%)]	Loss: 2.264426
Train Epoch: 3 [640/3000 (21%)]	Loss: 2.223513
Train Epoch: 3 [1280/3000 (43%)]	Loss: 2.149489
Train Epoch: 3 [1920/3000 (64%)]	Loss: 2.193071
Train Epoch: 3 [2560/3000 (85%)]	Loss: 2.074207

Test set: Average loss: 1.9471, Accuracy: 4607/10000 (46%)

Train Epoch: 4 [0/3000 (0%)]	Loss: 1.978141
Train Epoch: 4 [640/3000 (21%)]	Loss: 2.027267
Train Epoch: 4 [1280/

### Training

In [57]:
train_unl_loader.batch_size
len(train_unl_loader.dataset.train_data)

47000

In [65]:
list(enumerate(rec_unl_loader))[0]

(0, 
 ( 0 ,.,.) = 
   0.5542  0.4120  0.4187  ...   0.7066  0.4529  0.4194
 
 ( 1 ,.,.) = 
   0.5716  0.4887  0.3700  ...   0.7167  0.4359  0.4321
 
 ( 2 ,.,.) = 
   0.5621  0.4437  0.3946  ...   0.7014  0.4548  0.4605
 ... 
 
 (61 ,.,.) = 
   0.6282  0.5878  0.3196  ...   0.5776  0.5457  0.5419
 
 (62 ,.,.) = 
   0.4259  0.4468  0.3233  ...   0.7559  0.5211  0.4210
 
 (63 ,.,.) = 
   0.6111  0.4168  0.3968  ...   0.6848  0.4354  0.4241
 [torch.FloatTensor of size 64x1x784])

In [104]:
for idx, (data,target) in enumerate(train_unl_loader):
    print(data[1])
    break


(0 ,.,.) = 

Columns 0 to 8 
  -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.1824
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242  1.7269
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.0933  2.2105
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242  1.8923  2.7833
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242  0.4286  2.6306  2.7833
 -0.4242 -0.4242 -0.

In [105]:
for idx, data in enumerate(rec_unl_loader):
    print(data[1])
    break


(0 ,0 ,.,.) = 

Columns 0 to 8 
   0.5335  0.4836  0.2274  0.3537  0.3119  0.5831  0.4096  0.4572  0.5225
  0.2490  0.6028  0.4378  0.4671  0.6842  0.6516  0.2662  0.5216  0.4212
  0.5561  0.5995  0.5136  0.6251  0.4179  0.6973  0.4954  0.5705  0.3481
  0.4655  0.6654  0.6571  0.6203  0.4649  0.6675  0.3329  0.5135  0.5596
  0.3470  0.5391  0.3722  0.5539  0.6951  0.3925  0.7076  0.6224  0.6059
  0.5418  0.6224  0.4284  0.4600  0.5903  0.6538  0.3930  0.5804  0.3597
  0.5174  0.5734  0.5511  0.6694  0.3567  0.5176  0.5208  0.4144  0.5192
  0.5168  0.4466  0.3260  0.4771  0.2681  0.4995  0.4555  0.5913  0.4616
  0.4116  0.3095  0.5996  0.4478  0.4256  0.5943  0.4866  0.4326  0.6170
  0.3752  0.5098  0.4847  0.4071  0.4758  0.5885  0.4319  0.5428  0.4881
  0.4813  0.3900  0.6634  0.4956  0.4745  0.4345  0.6089  0.3543  0.4658
  0.4598  0.4560  0.3598  0.5576  0.3632  0.4484  0.4777  0.3592  0.3565
  0.5236  0.3542  0.6495  0.5818  0.6323  0.6409  0.3705  0.4180  0.4844
  0.5839  0.6169 

In [102]:
"""
semi-supervised learning
"""
time = 0

def alpha(t):
    alpha_f = 3.
    T1 = 100
    T2 = 600
    if t < T1:
        return 0
    elif t < T2:
        return alpha_f * (t-T1) / (T2-T1)
    else:
        return alpha_f

def output2p_label(output):
    """
    get the pseudo label from the model output 
    
    @param output: a Variable
    @return indices: a Variable, squeezed
    """
    y_i, indices = torch.max(output, 1)
    indices = indices.squeeze()
    indices = Variable(indices.data)
    return indices
    
def train_semi(epoch):
    global time
    model.train()
    
    n_labeled = len(train_loader.dataset.train_data)
    n_unlabeled = len(train_unl_loader.dataset.train_data)
    labeled_batch_size = train_loader.batch_size
    unlabeled_batch_size = train_unl_loader.batch_size

    # because #label < #unlabel, use round-robin for labeled data
    labeled_batches = int(np.ceil(1. * n_labeled / labeled_batch_size))
    batches = int(np.ceil(1. * n_unlabeled / unlabeled_batch_size))

    # locally allocate memory to store labeled data list, reduce overhead
    label_list = list(enumerate(train_loader))

    # unlabeled data has no label, enumeration causes error
    # so manually add 0's for convenience [ignore the label anyway]
    train_unl_loader.dataset.train_labels = torch.zeros(n_unlabeled)
    unlabel_list = list(enumerate(rec_unl_loader))
    
    # semi-supervised learning both labeled and unlabeled data
    for batch_id in range(batches):
        optimizer.zero_grad()
        time += 1
        
        # get labeled data to supervised learn
        idx, (data, target) = label_list[batch_id % labeled_batches]
        data, target = Variable(data), Variable(target)
        output = model(data)
        labeled_loss = criterion(output, target)

        # get unlabeled data to unsupervised learn
        idx, data = unlabel_list[batch_id]
        data = Variable(data)
        output = model(data)
        pseudo_label = output2p_label(output)
        unlabeled_loss = criterion(output, pseudo_label)
        
        # sum the losses
        loss = labeled_loss + alpha(time) * unlabeled_loss
        loss.backward()
        optimizer.step()

        
        if batch_id % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_id * len(data), len(train_unl_loader.dataset),
                100. * batch_id / len(train_unl_loader), loss.data[0]))
    

In [103]:
"""
for test and debugging
"""

for epoch in range(1, 20):
    train_semi(epoch)
    test(epoch, valid_loader)



IndexError: tuple index out of range

# Create Sample Submission

In [19]:
testset = pickle.load(open("test.p", "rb" ))

In [20]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)

In [21]:
testset.test_labels


 7
 2
 1
⋮ 
 4
 5
 6
[torch.LongTensor of size 10000]

## Test Accuuracy

In [22]:
# test(1, test_loader)

In [23]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    label_predict = np.concatenate((label_predict, temp))

In [24]:
label_predict

array([ 7.,  2.,  1., ...,  4.,  5.,  6.])

In [25]:
label_true = test_loader.dataset.test_labels.numpy()

In [26]:
diff_array = label_true - label_predict

In [27]:
len(np.where(diff_array != 0)[0])

254

In [28]:
import pandas as pd
true_label = pd.DataFrame(label_true, columns=['label'])
true_label.reset_index(inplace=True)
true_label.rename(columns={'index': 'ID'}, inplace=True)

In [29]:
true_label.head()

,ID,label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4


In [30]:
predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

In [31]:
predict_label.head(10)

,ID,label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4
5,5,1
6,6,4
7,7,9
8,8,5
9,9,9


In [32]:
predict_label.to_csv('sample_submission.csv', index=False)
true_label.to_csv('true_label.csv', index=False)